<a href="https://colab.research.google.com/github/SoonerTuran/DNVA/blob/main/DNVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

In [119]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [120]:
class ALU(nn.Module):
    def __init__(self, input_size, opcode_size, output_size, flag_size, hidden_size=32):
        super(ALU, self).__init__()

        # Layers definition
        self.linear1 = nn.Linear(self.input_size + self.opcode_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, self.output_size + flag_size)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        return x

In [121]:
class Decoder(nn.Module):
    def __init__(self, input_size):
        super(Decoder, self).__init__()

        output_size = 2 ** input_size
        self.fc1 = nn.Linear(input_size, input_size*2)
        self.fc2 = nn.Linear(input_size*2, input_size*3)
        self.fc3 = nn.Linear(input_size*3, input_size*4)
        self.fc4 = nn.Linear(input_size*4, input_size*5)
        self.fc5 = nn.Linear(input_size*5, output_size)

    def forward(self, x):
      x = torch.sin(self.fc1(x))
      x = torch.sin(self.fc2(x))
      x = torch.sin(self.fc3(x))
      x = torch.sin(self.fc4(x))
      x = torch.softmax(self.fc5(x), dim=1)
      return x

In [122]:
class RAM(nn.Module):
    def __init__(self, address_size, data_size):
        super(RAM, self).__init__()

        self.address_size = address_size
        self.data_size = data_size
        self.decoder = Decoder(address_size)

        # initialize a memory block
        self.memory = torch.randn(2**address_size, data_size)

    def forward(self, address, data, write):
        # address should be in one-hot representation
        one_hot_address = self.decoder(address)

        write = write.view(-1, 1)

        read_data = torch.matmul(one_hot_address, self.memory) * (1 - write)

        self.memory = (self.memory * (1 - one_hot_address.unsqueeze(-1) * write)) + data * one_hot_address.unsqueeze(-1) * write

        return read_data + data * write

In [123]:
def test_RAM():
    # Create a RAM instance with 2 bit address and 2 bit data
    address_size = 2
    data_size = 2
    ram = RAM(address_size, data_size)

    # Create some test data to write to the RAM
    test_data = torch.rand((1, data_size))
    print("Test Data: ", test_data)

    # Create a test address to write to/read from
    test_address = torch.randint(0, 2, (1, address_size))
    test_address = test_address.float()
    print("Test Address: ", test_address)

    print("Memory: ", ram.memory)
    # Write operation
    write_flag = torch.tensor([1.])  # write = 1
    ram(test_address, test_data, write_flag)
    print("Memory after write: ", ram.memory)

    # Read operation
    write_flag = torch.tensor([0.])  # write = 0
    read_data = ram(test_address, test_data, write_flag)
    print("Memory after read: ", ram.memory)

    print("Written data: ", test_data)
    print("Read data: ", read_data)

# Run the test
test_RAM()


Test Data:  tensor([[0.9350, 0.6063]])
Test Address:  tensor([[0., 0.]])
Memory:  tensor([[ 0.0795, -0.0887],
        [ 0.0410, -0.3014],
        [-1.2327, -0.4292],
        [ 1.2684, -0.6368]])
tensor([[4.1879e-29, 5.1141e-13, 1.0000e+00, 1.5383e-25]],
       grad_fn=<SoftmaxBackward0>)
Memory after write:  tensor([[[ 0.0795, -0.0887],
         [ 0.0410, -0.3014],
         [-0.1489,  0.0885],
         [ 1.2684, -0.6368]]], grad_fn=<AddBackward0>)
tensor([[4.1879e-29, 5.1141e-13, 1.0000e+00, 1.5383e-25]],
       grad_fn=<SoftmaxBackward0>)
Memory after read:  tensor([[[ 0.0795, -0.0887],
         [ 0.0410, -0.3014],
         [ 0.3931,  0.3474],
         [ 1.2684, -0.6368]]], grad_fn=<AddBackward0>)
Written data:  tensor([[0.9350, 0.6063]])
Read data:  tensor([[[0.3931, 0.3474]]], grad_fn=<AddBackward0>)


In [72]:
import itertools

input_size = 13
num_samples = 2 ** input_size
output_size = 2 ** input_size

# Tüm olası ikili sayıları oluşturmak için itertools.product kullanılıyor.
binary_values = list(itertools.product([0, 1], repeat=input_size))
# Toplam ikili sayı sayısı num_samples'tan fazla ise, sadece ilk num_samples kadarını alırız.
binary_values = binary_values[:num_samples]

# Liste halindeki ikili sayıları tensörlere dönüştürme
train_inputs = torch.tensor(binary_values, dtype=torch.int, device=device , requires_grad=False)
test_inputs = torch.randint(0, 2, (num_samples, input_size), device=device, requires_grad=False)

# One-hot çıktıları oluşturma
train_outputs = torch.zeros(num_samples, output_size, device=device)
test_outputs = torch.zeros(num_samples, output_size, device=device)
for i in range(num_samples):
    # Her ikili sayıyı ondalık olarak dönüştürme
    decimal1 = int(''.join(map(str, train_inputs[i].tolist())), 2)
    decimal2 = int(''.join(map(str, test_inputs[i].tolist())), 2)
    # Karşılık gelen indekse 1 atama
    train_outputs[i][decimal1] = 1
    test_outputs[i][decimal2] = 1

train_inputs = train_inputs.float()
test_inputs = test_inputs.float()
train_outputs = train_outputs.float()
test_outputs = test_outputs.float()

del binary_values
torch.cuda.empty_cache()

In [73]:
model = Decoder(input_size=input_size)
model.to(device)

# Setup a loss function
loss_fn = nn.CrossEntropyLoss()

# Setup an optimizer (stochastic gradient descent)
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

In [74]:
def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred.data, 1)
    correct = (predicted == torch.argmax(y_true, 1)).sum().item()
    accuracy = correct / y_true.size(0)
    return accuracy


epochs = 20000

# Track different values
epoch_count = []
loss_values = []
test_loss_values = []
accuracy_values = []
test_accuracy_values = []

#Training
for epoch in range(epochs):
  # Set the model to training mode
  model.train() # train mode in PyTorch sets all parameters that require gradients to require gradients

  # 1. Forward pass
  y_pred = model(train_inputs)

  # 2. Calculate the loss
  loss = loss_fn(y_pred, train_outputs)

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. Perform backpropagation on the loss with respect to the parameters of the model (calculate gradients of each parameter)
  loss.backward()

  # 5. Step the optimizer (perform gradient descent)
  optimizer.step() # by default how the optimizer changes will acculumate through the loop so... we have to zero them above in step 3 for the next iteration of the loop

  # Calculate training accuracy
  accuracy = calculate_accuracy(y_pred, train_outputs)

  ### Testing
  model.eval() # turns off different settings in the model not needed for evaluation/testing (dropout/batch norm layers)
  with torch.inference_mode(): # turns off gradient tracking & a couple more things behind the scenes
    # 1. Do the forward pass
    test_pred = model(test_inputs)

    # 2. Calculate the loss
    test_loss = loss_fn(test_pred, test_outputs)

    # Calculate testing accuracy
    test_accuracy = calculate_accuracy(test_pred, test_outputs)

  # Print out what's happenin'
  if epoch % 100  == 0:
    epoch_count.append(epoch)
    loss_values.append(loss.detach().item())
    test_loss_values.append(test_loss.detach().item())

    accuracy_values.append(accuracy)
    test_accuracy_values.append(test_accuracy)

    print(f"Epoch: {epoch} | Loss: {loss.detach().item():.6f} | Test loss: {test_loss.detach().item():.6f} | Train Accuracy: {accuracy:.6f} | Test Accuracy: {test_accuracy:.6f}")

  del loss, test_loss, y_pred
  torch.cuda.empty_cache()



tensor([[1.0611e-04, 1.1090e-04, 1.0229e-04,  ..., 1.1024e-04, 1.1210e-04,
         1.1799e-04],
        [1.0767e-04, 1.1254e-04, 1.0015e-04,  ..., 1.0859e-04, 1.0977e-04,
         1.1893e-04],
        [1.0411e-04, 1.0943e-04, 1.0289e-04,  ..., 1.0749e-04, 1.1221e-04,
         1.1306e-04],
        ...,
        [9.5136e-05, 1.0003e-04, 1.0913e-04,  ..., 1.0929e-04, 1.1410e-04,
         1.1469e-04],
        [9.4526e-05, 9.8809e-05, 1.0997e-04,  ..., 1.0852e-04, 1.1533e-04,
         1.1166e-04],
        [9.4059e-05, 9.9667e-05, 1.0825e-04,  ..., 1.0810e-04, 1.1391e-04,
         1.1198e-04]], grad_fn=<SoftmaxBackward0>)
tensor([[9.1245e-05, 9.7906e-05, 1.1107e-04,  ..., 1.1003e-04, 1.1492e-04,
         1.1283e-04],
        [9.0577e-05, 9.8757e-05, 1.0886e-04,  ..., 1.1045e-04, 1.1229e-04,
         1.0706e-04],
        [9.7870e-05, 1.0344e-04, 1.0586e-04,  ..., 1.0747e-04, 1.1271e-04,
         1.0580e-04],
        ...,
        [9.5003e-05, 9.9080e-05, 1.0888e-04,  ..., 1.0753e-04, 1.1426e-0

KeyboardInterrupt: ignored

In [ ]:
# Plot the loss curves
plt.plot(epoch_count, np.array(torch.tensor(loss_values)), label="Train loss")
plt.plot(epoch_count, np.array(torch.tensor(test_loss_values)), label="Test loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend();
plt.show()

# Plot the loss curves
plt.plot(epoch_count, np.array(torch.tensor(accuracy_values)), label="Train Acc")
plt.plot(epoch_count, np.array(torch.tensor(test_loss_values)), label="Test Acc")
plt.title("Training and test acc curves")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend();
plt.show()

In [ ]:
model.eval()
with torch.inference_mode():
  test_pred = model(test_inputs)

index = torch.randint(low=0, high=512, size=(1,1))[0][0]
print(test_inputs[index])

print(test_pred[index].round(decimals=2))
print(test_pred[index].round(decimals=2).argmax())

# Binary tensoru bir stringe dönüştürelim
binary_str = ''.join([str(int(i)) for i in test_inputs[index]])

# Binary stringi desimal sayıya dönüştürelim
decimal_number = int(binary_str, 2)
print(decimal_number)